In [ ]:
import ee

ee.Initialize()

In [ ]:
# load our features
features = ee.FeatureCollection("<Feature Collection Name Here>")
train = features.filter(ee.Filter.eq('type', 1))
test = features.filter(ee.Filter.eq('type', 2))

# create the model
from cnwi.cnwilib.random_forest import SmileRandomForest

model = SmileRandomForest()

# train the model
## load the predictors
with open('predictors.txt', 'r') as f:
    predictors = f.read().splitlines()

model.fit(
    features=train,
    classProperty='class_name',
    inputProperties=predictors,
)

# predict the model on the test set
predict = model.predict(test)

# save the model
model.save_model()

# save the assessment metrics
from cnwi.cnwilib.assessment import AssessmentTable

assessment = AssessmentTable(
    predicted=predict,
    actual='class_name',
    order=features.aggregate_array('class_name').distinct()
)

table = assessment.create_metrics()

## export to drive
task = ee.batch.Export.table.toDrive(
    collection=table,
    description='assessment',
    folder='cnwi-assessment-metrics',
    fileFormat='CSV'
)